In [1]:
using HDF5
using BenchmarkTools
using Revise 
using OwnTime
using Profile
using TimerOutputs

In [2]:
include("core.jl")
using .Kmeans1D

In [3]:
#path = joinpath(homedir(), "TFM", "ann-benchmarks",  "sift-128-euclidean.hdf5")
path = joinpath(homedir(), "Datasets", "SIFT1M", "sift-128-euclidean.hdf5")

f = h5open(path, "r")
X_tr_vecs = read(f["train"])

128×1000000 Matrix{Float32}:
  0.0   14.0    0.0   12.0   1.0   48.0  …   59.0   3.0  30.0    0.0  114.0
 16.0   35.0    1.0   47.0   1.0   69.0      17.0  12.0  12.0    5.0   31.0
 35.0   19.0    5.0   14.0   0.0    9.0      16.0   2.0  12.0   12.0    0.0
  5.0   20.0    3.0   25.0   0.0    6.0      34.0   0.0  15.0    0.0    0.0
 32.0    3.0   44.0    2.0  14.0    2.0       4.0   1.0   5.0    0.0    0.0
 31.0    1.0   40.0    3.0  16.0    3.0  …    1.0  26.0   2.0   54.0    0.0
 14.0   13.0   20.0    4.0  30.0    7.0       0.0  32.0   0.0  115.0    0.0
 10.0   11.0   14.0    7.0  50.0   25.0       2.0   3.0   8.0    0.0   10.0
 11.0   16.0   10.0   14.0   2.0   64.0       0.0  15.0   6.0    0.0  129.0
 78.0  119.0  100.0  122.0  40.0  130.0       9.0  46.0  18.0    4.0   48.0
 55.0   85.0   63.0   90.0  81.0   33.0  …  102.0  34.0  24.0   13.0    5.0
 10.0    5.0    7.0    7.0  25.0    4.0     121.0   0.0  19.0    2.0   13.0
 45.0    0.0   44.0    0.0  65.0    0.0       9.0   0.0   5

In [4]:
X = X_tr_vecs[1,1:1000]
k = 32
Kmeans1D.cluster_julia(X,k)

(clusters = [1, 8, 1, 7, 1, 17, 1, 8, 5, 10  …  2, 6, 21, 4, 3, 1, 1, 15, 7, 10], centroids = [0.33333333333333337, 2.482352941176472, 4.355932203389829, 6.5, 8.419354838709676, 10.488372093023257, 12.477272727272728, 15.06896551724138, 17.8125, 21.151515151515152  …  78.83333333333334, 83.91666666666667, 89.14285714285715, 96.28571428571429, 100.57142857142857, 107.25, 115.4, 119.99999999999999, 127.8, 136.0])

In [5]:
include("_core_trans.jl")
using ._Kmeans1D

centroids, clusters = _Kmeans1D.cluster(X, length(X), k)

([0.33333333333333337, 2.482352941176472, 4.355932203389829, 6.5, 8.419354838709676, 10.488372093023257, 12.477272727272728, 15.06896551724138, 17.8125, 21.151515151515152  …  78.83333333333334, 83.91666666666667, 89.14285714285715, 96.28571428571429, 100.57142857142857, 107.25, 115.4, 119.99999999999999, 127.8, 136.0], [1, 8, 1, 7, 1, 17, 1, 8, 5, 10  …  2, 6, 21, 4, 3, 1, 1, 15, 7, 10])

In [6]:
@code_warntype _Kmeans1D.cluster(X, length(X), k)

MethodInstance for Main._Kmeans1D.cluster(::Vector{Float32}, ::Int64, ::Int64)
  from cluster(array, n, k) in Main._Kmeans1D at /Users/dbuchaca/personal/git_stuff/julia_tutorials/optimizing_code/kmeans1d/_core_trans.jl:117
Arguments
  #self#::Core.Const(Main._Kmeans1D.cluster)
  array::Vector{Float32}
  n::Int64
  k::Int64
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  clusters::Vector{Int64}
  n_::Int64
  k_@_11::Int64
  t::Int64
  sorted_clusters::Vector{Int64}
  centroids::Vector{Float64}
  row_argmins::Union{Dict{Any, Any}, Dict{Int64, Int64}}
  C::Matrix{Float64}
  T::Matrix{Int64}
  D::Matrix{Float64}
  cost_calculator::Main._Kmeans1D.CostCalculator
  sorted_array::Vector{Float64}
  undo_sort_lookup::Vector{Int64}
  sort_idx::Vector{Int64}
  i@_23::Int64
  i@_24::Int64
  @_25::Union{Nothing, Tuple{Int64, Int64}}
  k_@_26::Int64
  min::Any
  argmin:

In [7]:
@benchmark Kmeans1D.cluster_julia(X, k)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  3.165 s …  3.167 s  ┊ GC (min … max): 3.66% … 3.41%
 Time  (median):     3.166 s             ┊ GC (median):    3.53%
 Time  (mean ± σ):   3.166 s ± 1.682 ms  ┊ GC (mean ± σ):  3.53% ± 0.18%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.16 s        Histogram: frequency by time        3.17 s <

 Memory estimate: 1.41 GiB, allocs estimate: 93307738.

In [8]:
#@profile Kmeans1D.cluster_julia(X,k)

In [9]:
#aux = Kmeans1D.cluster_julia(X,k)

In [10]:
#owntime()

In [11]:
#totaltime(stackframe_filter=filecontains("_core_trans.jl"))

In [12]:
#totaltime()

In [13]:
#Profile.print(format=:flat)